In [40]:
# We want a recent version with numpy.typing
%pip install -U numpy
  
%pip install -U scipy
%pip install -U numba

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/local/opt/python@3.9/bin/python3.9 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/local/opt/python@3.9/bin/python3.9 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 2.2 MB 1.2 MB/s 
     |████████████████████████████████| 18.5 MB 560 kB/s 
You should consider upgrading via the '/usr/local/opt/python@3.9/bin/python3.9 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [1]:
from enum import Enum
import numpy as np
from typing import Dict, List, Optional, Tuple
from numpy.typing import NDArray
from scipy.sparse import csr_matrix, spmatrix


# Type Aliases

For the python examples, we want to exploit mypy type checking and be
able to provide good type choices for numba `@jit`, so we specify exact numpy
types like `np.int64` rather than Python `int`s. For shorthand, these
are abstracted away as `T` for floating point scalar types, an `id` for
cases where we use ints as ids.

In [7]:
# Type Aliases

# For shorthand, we refer to the floating point type used to represent 
# scalar values as T, which allows easy type checking using mypy. This
# also allows us to easily switch to, e.g. np.float32
T = np.float64

# Many items in this 3D code use an array of 3 Ts, so we create a type
# shorthand for it 
T3 = NDArray[T]

# And also a convenience method to init a T3
def allocateT3() -> T3:
    return np.zeros((3,), dtype=T)

# Representing ids (pointers?). We could switch to int64 if that made sense
id = np.int32



# Data structures

## The mesh

We make a distinction between the data structures we use for loading a mesh from disk, and the
computational mesh (variables) that the program operates on. For MiniCombust, we do not prescribe
the former (in fact, we recommend leveraging existing, tested implementations like libmesh that are
compatible with gmsh).

For the latter, we allow variation to test the effect of, say SoA vs AoS layout. PETSC-FUN3D

In general, meshes can represent variables in terms of cells and faces, or in terms of nodes (vertices) and edges
(links).
It's possible to have staggered approaches where a variable of interest is defined at the cell centre,
but flux variables are at nodes.

For our problem, we use

We consider only the 3D case for MiniCombust.


Some common things are precalculated

Faces have areas
Cell have volumes

Cell faces are locally numbered clockwise
```
Map cell -> faces
Map face -> adjacent cells
Map cell -> neighbours (for particle tracking)
Map cell -> vertices

List mesh.regions

```

The mesh also has regions with ids


## Representation for calculations
[BASED ON dolfyn.90 READGEOMETRY]
* For every cell we store its face (ids) and nodes [WHAT DO WE NEED NODAL VALUES FOR?]
* For every face we store its adjacent cell (ids)


Note that loading meshes, partitioning them and pre-calculating these properties of cells and volumes
are all uninteresting from a scaling point of view: they're a once-off upfront cost. It's feasible that
this would even be pre-processing before a 'run' of the simulation.  However, we measure the total 
up-front cost, in case this is smething that needd to be optimised.

We don't even specify that these calculations needs to happen in parallel for MiniCombust.

The long-running bit is the simulation.



## Distributed meshes

We use 'domain decomposition'

### Partitioning
Partitioning libraries like Parmetis partition graphs defined in terms of nodes and edges. 
The graph we want to partition in this case is not the same as the nodes and edges that make up the 
computational mesh. It's a description of how different cells are adjacent to each other.

Here's an example:
[SHOW A 3D MESH WITH CELLS AND FACES AND ITS CORRESPONDING GRAPH]

### Representing ghost cells

## Are the cells for the FVM the same as the cells for the particles?

## Assembling a linear system from the mesh
We'll use the industry standard PETSc to represent matrices and vectors for linear solver.
Ideally we want zero-copy (DLPack)?

How to go from mesh variables to the assembled matrix.
Sparse matrix, dense vector.
Possible for this to be matrix-free Jacobian, but maybe we just use PETSc.



An unstructured mesh consists of polyhedral cells. We can define these by the edges (vertexes), cells and the
faces (adjacent between two cells). See fig:

[Fig 3.1 a from https://www.ctcms.nist.gov/fipy/download/fipy-1.2.3.pdf]


https://scicomp.stackexchange.com/questions/4733/what-are-some-good-data-types-for-unstructured-cell-centered-fvm-cfd-code

MiniCombust is face-based, i.e. Cell Centered FVM (CC-FVM)

We want a distributed mesh where we can find the neighbouring faces

??

In FVM, we are concerned with the average value of a variable of interest within a control volume (i.e. cell). Thiis
average value is stored at the cell centre. Fluxes for a face are approximated using the values from the two cells adjacent to the face. This is a low-order approximation, but is simple to store and results in systems with low-bandwidth matrices.

We use co-located rather than staggered grids for pressure and velocity/heat?

Mesh elements (cells, faces, edges) have Globally unique IDs (over distributed memories)
Each element may also have a local ID, that is not globally unqiue.
This allows renumbering locally for more efficient data structure access.
Cells are grouped into locally owned an ghost cells


The computational mesh consists of polyhedral cells, the faces of which have maximum 4 vertices

Face connectivity is also stored as CSR for easy traversal.

Vertex connectivity connectivity is stored as a distributed adjacency list:

CSR?
 CSR xadj  vs adjncy
For each node we know whether it is local or ghost
For ghost nodes, we know the process it belongs to

Since is store as CSR, we can always trivially look up neighbour nodes of a cell based on it's position in CSR
Describe

We partitional the cells by building a graph where nodes are the cell ID and edges define adjaccent cells


Nodes are totally a thing. We rely on a nodes/edges thing to partition mesh and also what does CSR mean if not
nodes and edges.

(we use process ID rather than rank, but you can think of it as the same fr MPI)

In contrast to NetworkX (edges, vertices), meshes are defined by their faces.
IS THIS TRUE?

The canonical app uses libmesh or something instead of implementing this from scratch?


Inspired by the PETSC-FUN3D papers (https://www.mcs.anl.gov/research/projects/petsc-fun3d/Papers/sc00.pdf) we
use AoS ("Interlacing"), CSR, Structural Blocking, Edge Reordering (Reverse Cuthill-McKee)
But actually want MPI-OpenMP (Maybe?)

## Partitioning the mesh over nodes
Uses Parmetis to partition the mesh for a number of ranks.
Use ParMetis 
Using a partitioner such as parmetis or metis, the nodes of the mesh are partitioned 
What does this mean for faces and cells?
Who owns a face variable?


## Sparse matrices
Each timestep results in a linear system Ax = b where A is sparse.
How do we represent these?
Are we matrix free?

## Timesteps, iterations and sweeps
Timestep: delta T as system evolves
Sweeps: we improve non-linear PDE accuracy by running several sweeps of the solution for each timestep, subsituting the
previous solution value at the same time step value
Iterations: number of iterations of linear solver


## Boundary type
Inlet, Outlet, wall, symmety planes?


```Fortran
////
 type :: CellData
     real    :: x(3)                      ! center coordinates
     real    :: vol                       ! volume
     integer :: ctid = 1                  ! fluid type id as set in fluid table
   end type

///
```


## Particles
```Fortran
   type ParticleProp
     real    :: x0(3) = 0.0                        ! starting coordinates
     real    :: v0(3) = 0.0                        ! starting velocity
     real    :: x1(3) = 0.0                        ! current coordinates
     real    :: v1(3) = 0.0                        ! current velocity
     real    :: a1(3) = 0.0                        ! current acceleration
     integer :: cell0 =  0                         ! starting cell the particle is in
     integer :: cell1 =  0                         ! current cell the particle is in
     integer :: face  =  0                         ! current face the particle is on
     real    :: dens0 = -1.0  ! <= undefined flag  ! current density
     real    :: diam0 =  1.0
     real    :: mass0 =  1.0
     logical :: wall  = .false.
   end type
```


https://www.cfd-online.com/Wiki/Discretization_of_the_diffusion_term


```
// Read number of cells
read ScaleFactor [detail, ignore]
allocate the Cell array (each has 3d coords, vol and fluid type id)


for each cell
    fill in coords and vol, fuel type
apply scale factor to all geom components and volume

read maxfaces [maximum facces of any cell]
allocate NFaces array (int number of faces for each cell)
allocate CFace array (list of faces)

```

### Questions
Does each processor need to kmnow more that its global-local mappings (i.e. nobody needs the total mapping?)

In the design, can we use a master/workers approach so that the partitioning etc. is done in the master node (maybe in parallel?) and then the workers are started with their bits, already renumbered?



In [6]:
class FluidType(Enum):
    SOLID = 0
    FLUID = 1
    SYMMETRY_PLANE = 2
    WALL = 3
    PRESSURE = 4
    CYCLIC = 5
    DOMAIN_BOUNDARY = 6

class BoundaryType(Enum):
    WALL = 1
    INLET = 2
    OUTLET = 3
    PRESSURE = 4
    SYMMETRY_PLANE = 5
    THIN = 6
    CYCLIC = 7
    DOMAIN_BOUNDARY = 8

# The bits of the A matrix
class CoefficientTerms:
    r: NDArray[T] # reciprocal coefficients (for pressure correction), indexed by cell id 
    u: NDArray[T] # u component indexed by cell id
    v: NDArray[T] # v component indexed by cell id
    w: NDArray[T] # w component indexed by cell id

# The bits of the S matrix
class SourceTerms:
    u: NDArray[T] # u component indexed by cell id
    v: NDArray[T] # v component indexed by cell id
    w: NDArray[T] # w component indexed by cell id

class Cell:
    x: T3 = allocateT3()
    volume: T
    fluid_type: FluidType

class Face:
     boundary_id: id
     first_cell_id: id
     second_cell_id: id
     vertex_ids: list[id]
     area: T
     normal_components: T3 = allocateT3()
     centre_coords: T3 = allocateT3()
     interpolation_factor: T # $\lambda$
     rlencos: T # impl. coeff.: area/|Xpn|/vect_cosangle(n,Xpn)
     xnac: T3 = allocateT3() # auxillary vectors TODO what is this?
     xpac: T3 = allocateT3() # TODO what is this?

class Boundary:
     face_id: id                            # Id of the face this boundary belongs to 
     vertex_ids: list[id]           
     region_id : id                         # region id as set in rtable TODO what is rtable
     distance_to_cell_centre : T3      # normal distance from cell face center to cell center
     yplus: T                        # y+ TODO what is this?
     uplus: T                        # u+ TODO what is this?
     shear_stress: T3                # shear stress components
     normal_stress: T3               # normal stress components
     heat_transfer_coeff: T          # h
     heat_flux: T                    # q (in W/m2)
     local_wall_temperature: T       # symbol T in equations

class BoundaryRegion:
    name: str
    region_type: BoundaryType
    velocity: T3 = allocateT3()     # uvw
    density: T = 1.205 
    temperature: T = 293.            # symbol T in eqs
    pressure : T = 0.                # P
    resistance: T = 0.               # R
    turbulent_kinetic_energy : T= 1.e-6        # k 
    turbulent_dissipation: T = 1.0             # \epsilon
    is_no_slip : bool = True                # slip/no_slip
    is_standard: bool = True                # standard/roughness
    elog: T = 9.                     # E-parameter
    ylog: T = 11.                    # + value where u+=y+ matches wall law
    roughness: T = 0.03              # z0         
    is_split_flow_rate: bool =  True        # split or fixed
    split_velocity: T = 1.0          # TODO what is this
    is_adiabatic: bool = True               # adiab or fixed temperature
    is_fixed_flux_temperature: bool = True  # fixed flux temperature
    num_boundaries_using_this_region: np.int32 = 0
    area: T

    total_mass_flow: T = 0.          # total mass flow in this region
    mass_flux_corr_factor: T  = 1.   # TODO correlation or correction???
    prescribed_mass_flow: T = 0.     # "mffixed"
    is_prescribed_flux_mass: T = False 
    is_table_of_profiles: T = False

    face_normal : T3 = allocateT3()
    face_tangent : T3 = allocateT3()

    is_perfect_matching : bool = True # Perfect matching or search
    central_point: T3 = allocateT3()
    direction: T3 = np.array((1., 0., 0.), dtype=T)
    translation: T3 = allocateT3()
    angle: T = 0. # rotation angle (=0.0 => translation)

class Geometry:
    # Can these actually just be len(struct?)
    num_vertices: np.int64
    num_cells: np.int64
    num_boundaries: np.int64
    num_regions: np.int64
    num_faces: np.int64
    num_int: np.int64 # ??? TODO what is this
    num_non_zero: np.int64
    num_outlets: np.int64

    split: T # TODO what is this?

    vertices: NDArray[T] # 2D Probably want to use scipy.sparse.csr_matrix instead? TODO What values are stored here?
    face_normals: NDArray[T] # 2D
    faces: list[Face]

    num_faces_per_cell: NDArray[np.int32] # number of faces for each cell (indexed by cell id)
    num_nodes_per_cell: NDArray[np.int32] # number of faces for each cell (indexed by node id)

    faceToCells: dict[Face, tuple[Cell, Cell]]
    cellToFaces: dict[Cell, list[Face]]

    cells: list[Cell]
    faces: list[Face]
    boundaries: list[Boundary]
    regions: list[BoundaryRegion]



In [9]:
class Particle:
    def __init__(self):
        self.starting_coords: T3 = allocateT3()
        self.starting_velocity: T3 = allocateT3()
        self.current_coords: T3 = allocateT3()
        self.current_velocity: T3 = allocateT3()
        self.current_acceleration: T3 = allocateT3()
        self.starting_cell_id : id
        self.current_cell_id : id
        self.current_face : id
        self.current_density : Optional[T]
        self.diameter : T
        self.mass: T
        self.is_wall: bool

class ParticleTrackData:
        def __init__(self):
            self.x : T3 = allocateT3()    # coordinates 
            self.v : T3 = allocateT3()    # velocity: np.ndarray[ScalarType] = np.zeros((3,), type=ScalarType)
            self.cell: id                 # cell the particle is in
            self.time: T                  # travelling time since start

class Particles:
    # The actual variables
    tracks: list[list[ParticleTrackData]] = []
    particles: list[Particle]
    tmp_particles: list[Particle]
